In [1]:
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from transformers import pipeline

In [2]:
pipe = pipeline("feature-extraction", model="microsoft/codebert-base")
df = pd.read_parquet("output_data/dataframe_text_embeddings.parquet")
qdrant = QdrantClient("localhost", port=6333)

In [20]:
df.head()

,text,vector
0,class Solution(object):\n def climbStairs(s...,"[-0.21761193871498108, -0.006907954812049866, ..."
1,"def calculate_area(height, l, r):\n return ...","[-0.17671486735343933, -0.11769969761371613, -..."
2,"class Solution:\n def rob(self, nums) -> in...","[-0.11747059971094131, -0.04676730930805206, -..."
3,# Definition for singly-linked list.\n# class ...,"[-0.21408772468566895, -0.02248210459947586, -..."
4,class Solution:\n def longestPalindrome(sel...,"[-0.35301244258880615, -0.0692468136548996, -0..."


In [3]:
collection_name = "code"
vector_len = len(df.iloc[0][1])
vector_len

768

In [4]:
try:
    qdrant.recreate_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vector_len, distance=Distance.COSINE),
        replication_factor=2,
        shard_number=6,
    )
except Exception as e:
    pass

In [6]:
batch_size = 100

for i in range(0, len(df.index), batch_size):
    pos_final = min(i + batch_size, len(df.index))

    chunk = df.iloc[i:pos_final]

    qdrant.upsert(
        collection_name=collection_name,
        points=[
            PointStruct(
                id=idx,
                vector=row["vector"].tolist(),
                payload={"answer": row["text"]},
            )
            for idx, row in chunk.iterrows()
        ],
    )